# Conversational AI - aka Chatbot!

Adding history of past conversations and Context or Prompt Enrichments

In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
#from IPython.display import Markdown, display
from openai import OpenAI
import json
from typing import List
import anthropic
import gradio as gr



In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Initialize

openai = OpenAI()
claude = anthropic.Anthropic()
MODEL = 'gpt-4o-mini'

In [ ]:
system_message = "You are a helpful assistant"

In [ ]:
#We will write a function `chat(message, history)` where:  
#**message** is the prompt to use  
#**history** is the past conversation, in OpenAI format  
#We will combine the system message, history and latest message, then call OpenAI.


def chat(message, history):
    #messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    messages = [{"role": "system", "content": system_message},
                {"role": "user", "content": message}]

    #print("History is:")
    #print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

# 🧠 How Memory Works in Chatbots

In conversational chatbots, memory is usually simulated by passing past interactions back into the model.

LLMs themselves are stateless → they don’t remember beyond a single request.

To give the illusion of memory, developers store past conversation history (in a list, DB, or vector store).

Each new API call includes this history as context so the model can generate a consistent, coherent reply.

👉 In short: memory is “fed back in,” not “remembered natively.”

In [ ]:
#Context enrichment
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

## Conversation Replay or Context-Window Memory or Naïve Chat Memory or Buffer Memory
### Example:
```
messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

```

How it works: Store all past messages → concatenate → send back with each request.

Pros: Simple, coherent, no extra infra.

Cons: Grows fast, hits token/context limits, gets expensive.

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [ ]:
#Prompt Chaining
print(system_message)

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
# I've also improved the structure of this function

def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()